In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.datasets import imdb 
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
# Load the imdb datasets 
max_features = 10000 # Vocabulary size 
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(25000,)

In [ ]:
# Inspect a sample review and its label 
sample_review = X_train[0]
sample_label = y_train[0]
word_index = imdb.get_word_index()
reverse_word_index = {val: key for key, val in word_index.items()}
# Skips the first 3 reserved words 
# In IMDB dataset, Keras reserves first 3 indices for special purpose
# Look up the word, if not found then use ?
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in sample_review])
decoded_review  

"? this film was just brilliant casting ? ? story direction ? really ? the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same ? ? as myself so i loved the fact there was a real ? with this film the ? ? throughout the film were great it was just brilliant so much that i ? the film as soon as it was released for ? and would recommend it to everyone to watch and the ? ? was amazing really ? at the end it was so sad and you know what they say if you ? at a film it must have been good and this definitely was also ? to the two little ? that played the ? of ? and paul they were just brilliant children are often left out of the ? ? i think because the stars that play them all ? up are such a big ? for the whole film but these children are amazing and should be ? for what they have done don't you think the whole story was so ? because it was true and was ? life after all that was ? with us all"

In [ ]:
from tensorflow.keras.preprocessing import sequence 
max_len = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_train

In [16]:
# Train simple RNN 
model = Sequential() 
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          128000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 161025 (629.00 KB)
Trainable params: 161025 (629.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Create an instance of EarlyStopping callback 
from tensorflow.keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
earlyStopping

In [19]:
# Train the model with Early stopping 
history = model.fit(
    X_train, y_train, epochs = 10, batch_size = 32, 
    validation_split = 0.2,
    callbacks=[earlyStopping]
)

Epoch 1/10


625/625 [==============================] - 56s 83ms/step - loss: 6043.4683 - accuracy: 0.5785 - val_loss: 0.6325 - val_accuracy: 0.6086
Epoch 2/10
625/625 [==============================] - 52s 82ms/step - loss: 0.5933 - accuracy: 0.6808 - val_loss: 0.5319 - val_accuracy: 0.7378
Epoch 3/10
625/625 [==============================] - 51s 82ms/step - loss: 0.5135 - accuracy: 0.7515 - val_loss: 0.4953 - val_accuracy: 0.7690
Epoch 4/10
625/625 [==============================] - 49s 79ms/step - loss: 0.7307 - accuracy: 0.6604 - val_loss: 0.6059 - val_accuracy: 0.6598
Epoch 5/10
625/625 [==============================] - 49s 79ms/step - loss: 0.5621 - accuracy: 0.7048 - val_loss: 0.5444 - val_accuracy: 0.7338
Epoch 6/10
625/625 [==============================] - 50s 80ms/step - loss: 0.5514 - accuracy: 0.7710 - val_loss: 0.5462 - val_accuracy: 0.7344
Epoch 7/10
625/625 [==============================] - 49s 79ms/step - loss: 0.4687 - accuracy: 0.7911 - val_loss: 0.4853 - val_accu

In [20]:
# Save model file 
model.save('simple_rnn_imdb.h5')

d:\Code\Python\ann-classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
